In [1]:
from scipy.stats import ttest_1samp
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


**Задача:** Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats

Считываем данные для анализа

In [3]:
df = pd.read_excel('D:\Lena\Geekbrains\Аналитика курс\АВ тестирование\Lesson 9\ABtest9.xlsx')
df.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [5]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


На данном этапе анализа мы видим, что возват игроков в целом на первый день почти 46 %, на седьмой уже меньше почти 19 %

In [6]:
df.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

In [7]:
v = df.\
    groupby('userid', as_index=False).\
    agg({'version': pd.Series.nunique})

In [8]:
v.head(10)

,userid,version
0,116,1
1,337,1
2,377,1
3,483,1
4,488,1
5,540,1
6,1066,1
7,1444,1
8,1574,1
9,1587,1


In [9]:
v.query('version > 1').count()

userid     0
version    0
dtype: int64

Игроков попавших одновременно в контрольную и тестовую группу нет

In [10]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

In [11]:
mannwhitneyu(df[df.version == 'gate_30'].sum_gamerounds,
             df[df.version == 'gate_40'].sum_gamerounds)

MannwhitneyuResult(statistic=1024331250.5, pvalue=0.05020880772044255)

pvalue очень блиско к 0.05 соответственно этого теста будит не достаточно для принятия решения

Проверим сколько игроков возвращаются к игре через 1 день по контройльной и тестовой группе

In [12]:
oneday = df.retention_1.groupby(df.version).sum()/df.retention_1.groupby(df.version).count()
oneday

version
gate_30    0.448188
gate_40    0.442283
Name: retention_1, dtype: float64

Мы видим очень маленько статистическое различие. Проверим этот показатель через 7 дней

In [13]:
sevenday = df.retention_7.groupby(df.version).sum()/df.retention_7.groupby(df.version).count()
sevenday

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Через 7 дней порог входа в игру больше в контрольной группе, но только на 0,8%

По этим данным можно сделать вывод, что вывод новой версии не поможет оставить пользователей в игре, поэтому оставляем прежнюю версию

In [14]:
from scipy.stats import shapiro

In [15]:
alpha = 0.05

st = shapiro(df.retention_1)
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



In [16]:
alpha = 0.05

st = shapiro(df.retention_7)
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



In [17]:
import scipy.stats as stats

In [18]:
control = df.query('version == "gate_30"')

test = df.query('version == "gate_40"')


In [19]:
allcontrol = control.shape[0]
allcontrol

44700

In [20]:
alltest = test.shape[0]
alltest

45489

In [21]:
controluser = control[(control['retention_1'] == 1) & (control['retention_7'] == 1)].shape[0]
controluser

6676

In [22]:
testuser = test[(test['retention_1'] == 1) & (test['retention_7'] == 1)].shape[0]
testuser

6506

Выясним сколько пользователей заходили и в первый день и седьмой по группам

In [23]:
users_add_control = controluser/allcontrol*100
users_add_control

14.935123042505595

In [24]:
users_add_test = testuser/alltest*100
users_add_test

14.302358812020488

In [25]:
dif = users_add_control-users_add_test
dif

0.6327642304851064

Мы видим, что в контройльной группе посещали игру и в первый и в седьмой день на 0,63% больше, что еще раз доказывает нецелесообразность выкатывать обновление 

In [26]:
control.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [27]:
test.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


По этим данным так же видно, что в контрольной группе возвращалось больше пользователей на седьмой день, что подтверждает правильность ранее сделанного вывода сохранения изначальной версии 